In [21]:
# %% ========================================
# 0. ライブラリ & データ読み込み
# ========================================
import pandas as pd
import numpy as np
import os

# 00 の出力（クリーンデータ）
CLEAN_PATH = "/workspace/data/00/StatsAllGroup_cleaned.parquet"

# 01_exploration の出力（選手プロファイル）
PLAYER_PROFILE_PATH = "/workspace/data/01/fp_player_final_categories.parquet"

df_clean = pd.read_parquet(CLEAN_PATH)
profiles = pd.read_parquet(PLAYER_PROFILE_PATH)

df_clean = df_clean[df_clean["position_group"] == "FP"].copy()
full_player_ids = profiles.loc[
    profiles["final_category"] == "full",
    "athlete_id"
].unique()

df_valid = df_clean[df_clean["athlete_id"].isin(full_player_ids)].copy()

print(f"✅ Loaded cleaned data: {df_clean.shape[0]} rows, {df_clean['athlete_name'].nunique()} athletes")
print(f"✅ Loaded player profiles: {profiles.shape[0]} rows")
print(f"🎯 Using FULL players only: {len(full_player_ids)} athletes")
print(f"   -> Filtered data: {df_valid.shape[0]} rows, {df_valid['athlete_name'].nunique()} athletes")



✅ Loaded cleaned data: 65471 rows, 65 athletes
✅ Loaded player profiles: 53 rows
🎯 Using FULL players only: 30 athletes
   -> Filtered data: 44034 rows, 32 athletes


In [22]:
print("\n🔍 データ型ごとの列名一覧:")

dtype_map = {}
for col, dtype in df_valid.dtypes.items():
    dtype_map.setdefault(str(dtype), []).append(col)

for dtype, cols in dtype_map.items():
    print(f"\n--- {dtype} ({len(cols)} columns) ---")
    for c in cols:
        print(" -", c)


🔍 データ型ごとの列名一覧:

--- datetime64[ns] (3 columns) ---
 - date_
 - start_dt
 - end_dt

--- float64 (50 columns) ---
 - session_duration_sec
 - start_time
 - end_time
 - total_duration
 - total_distance
 - max_heart_rate
 - mean_heart_rate
 - min_heart_rate
 - max_vel
 - velocity_band1_total_distance
 - velocity_band2_total_distance
 - velocity_band3_total_distance
 - velocity_band4_total_distance
 - velocity_band5_total_distance
 - velocity_band6_total_distance
 - gen2_velocity_band1_total_effort_count
 - gen2_velocity_band2_total_effort_count
 - velocity2_band6_total_distance
 - velocity2_band7_total_distance
 - velocity2_band8_total_distance
 - heart_rate_band2_total_duration
 - heart_rate_band3_total_duration
 - total_player_load
 - total_jumps
 - total_dive_load_centre
 - total_dive_load_left
 - total_dive_load_right
 - dive_right_count
 - dive_left_count
 - dive_centre_count
 - total_dives_centre
 - total_dives_left
 - total_dives_right
 - total_time_to_feet_centre
 - total_time_to_f

In [23]:
# ============================================================
# 日次集計（session → daily）
# ============================================================

df = df_valid.copy()
df["date_"] = df["date_"].dt.normalize()

# Match Day フラグ
df["is_match_day"] = df["activity_name"].str.contains(
    r"vs|VS|tm|TM|match|試合", case=False, na=False
).astype(int)

# 数値列の自動集計定義
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ["start_time", "end_time"]]

agg_dict = {}
for col in numeric_cols:
    if col == "mean_heart_rate":
        agg_dict[col] = "mean"
    elif col in ["max_heart_rate", "max_vel"]:
        agg_dict[col] = "max"
    elif col == "min_heart_rate":
        agg_dict[col] = "min"
    else:
        agg_dict[col] = "sum"

agg_dict["is_match_day"] = "max"

# チーム全体の Match Day
match_days = (
    df.loc[df["is_match_day"] == 1, "date_"]
      .drop_duplicates()
      .sort_values()
      .to_numpy()
      .astype("datetime64[D]")
)

# 日次集計（IDベース）
daily_raw = (
    df.groupby(["athlete_id", "date_"])
      .agg(agg_dict)
      .reset_index()
      .sort_values(["athlete_id", "date_"])
)

# athlete_id : athlete_name を 1:1 に固定
athlete_master = (
    df[["athlete_id", "athlete_name"]]
    .drop_duplicates(subset=["athlete_id"])
)

daily_raw = daily_raw.merge(
    athlete_master,
    on="athlete_id",
    how="left"
)

# md_offset（最も近い試合）
if len(match_days) == 0:
    daily_raw["md_offset"] = np.nan
else:
    dates = daily_raw["date_"].to_numpy().astype("datetime64[D]")
    diff = (dates[:, None] - match_days[None, :]).astype("timedelta64[D]").astype(int)
    daily_raw["md_offset"] = diff[np.arange(len(diff)), np.abs(diff).argmin(axis=1)]

# md_phase
def to_md_phase(x):
    if pd.isna(x): return "NONE"
    if x == 0: return "MD"
    if x == 1: return "MD+1"
    if x == -1: return "MD-1"
    if x == 2: return "MD+2"
    if x == -2: return "MD-2"
    return "OTHER"

daily_raw["md_phase"] = daily_raw["md_offset"].apply(to_md_phase)

print("✅ 日次集計完了:", daily_raw.shape)


✅ 日次集計完了: (11194, 66)


In [24]:
daily_raw = daily_raw.drop_duplicates(subset=["athlete_id", "date_"])


In [25]:
# ============================================================
# Season-aware ACWR（EWMA）
# ============================================================

def add_acwr_features_season_aware(daily_df, var, acute=7, chronic=28, break_days=30):
    df_acwr = daily_df.sort_values(["athlete_id", "date_"]).copy()

    df_acwr[f"EWMA{acute}_{var}"] = np.nan
    df_acwr[f"EWMA{chronic}_{var}"] = np.nan
    df_acwr[f"ACWR_EWMA_{var}"] = np.nan

    for athlete_id, idx in df_acwr.groupby("athlete_id").groups.items():
        sub = df_acwr.loc[idx].sort_values("date_")

        gap_days = sub["date_"].diff().dt.days
        season_block = (gap_days > break_days).cumsum().fillna(0).astype(int)

        for _, b_idx in sub.groupby(season_block).groups.items():
            s = df_acwr.loc[b_idx, var]
            if s.notna().sum() == 0:
                continue

            acute_series = s.ewm(span=acute, adjust=False).mean()
            chronic_series = s.ewm(span=chronic, adjust=False).mean()
            acwr_series = acute_series / chronic_series
            acwr_series.iloc[:chronic] = np.nan

            df_acwr.loc[b_idx, f"EWMA{acute}_{var}"] = acute_series
            df_acwr.loc[b_idx, f"EWMA{chronic}_{var}"] = chronic_series
            df_acwr.loc[b_idx, f"ACWR_EWMA_{var}"] = acwr_series

    return df_acwr



In [26]:
# ============================================================
# ACWR 対象変数
# ============================================================

target_vars = ["total_distance", "total_player_load"]

# gap_days 分布チェック
gap_examples = []
for athlete_id, sub in daily_raw.groupby("athlete_id"):
    g = sub.sort_values("date_")["date_"].diff().dt.days.dropna()
    if not g.empty:
        gap_examples.append(g)

if gap_examples:
    gaps = pd.concat(gap_examples)
    print(gaps.describe())
else:
    print("gap が計算できる選手がいません")

# ACWR 付与
for var in target_vars:
    print(f"⚙ ACWR 計算中: {var}")
    daily_raw = add_acwr_features_season_aware(daily_raw, var)

print("✅ ACWR / EWMA 特徴量追加完了")



count    11164.000000
mean         1.622358
std          3.614347
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        159.000000
Name: date_, dtype: float64
⚙ ACWR 計算中: total_distance
⚙ ACWR 計算中: total_player_load
✅ ACWR / EWMA 特徴量追加完了


In [27]:
# ============================================================
# 保存（日次特徴量：FP / full players）
# ============================================================

DAILY_DIR = "/workspace/data/02"
os.makedirs(DAILY_DIR, exist_ok=True)

daily_csv_path = f"{DAILY_DIR}/fp_full_players_daily.csv"
daily_parquet_path = f"{DAILY_DIR}/fp_full_players_daily.parquet"

# 一意性チェック
assert daily_raw[["athlete_id", "date_"]].duplicated().sum() == 0

daily_raw.to_csv(daily_csv_path, index=False)
daily_raw.to_parquet(daily_parquet_path, index=False)

print("💾 Saved daily data:")
print(" - CSV   :", daily_csv_path)
print(" - Parquet:", daily_parquet_path)



💾 Saved daily data:
 - CSV   : /workspace/data/02/fp_full_players_daily.csv
 - Parquet: /workspace/data/02/fp_full_players_daily.parquet


In [28]:
print("\n🔍 ACWR / EWMA 列")
acwr_cols = [c for c in daily_raw.columns if "ACWR_EWMA" in c or "EWMA7_" in c or "EWMA28_" in c]
print(acwr_cols)

display(daily_raw[["athlete_id", "athlete_name", "date_"] + acwr_cols].head(30))



🔍 ACWR / EWMA 列
['EWMA7_total_distance', 'EWMA28_total_distance', 'ACWR_EWMA_total_distance', 'EWMA7_total_player_load', 'EWMA28_total_player_load', 'ACWR_EWMA_total_player_load']


,athlete_id,athlete_name,date_,EWMA7_total_distance,EWMA28_total_distance,ACWR_EWMA_total_distance,EWMA7_total_player_load,EWMA28_total_player_load,ACWR_EWMA_total_player_load
0,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-12,7469.963550,7469.963550,NaN,814.200630,814.200630,NaN
1,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-13,7310.626227,7426.008427,NaN,795.414223,809.018173,NaN
2,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-14,7602.282476,7498.507930,NaN,847.145489,822.350664,NaN
3,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-16,8328.676322,7706.049304,NaN,900.640177,838.817807,NaN
4,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-17,8126.110984,7693.109005,NaN,876.273430,836.359557,NaN
5,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-18,8588.908083,7850.639376,NaN,928.891278,853.627507,NaN
6,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-20,8258.821170,7810.496000,NaN,863.449853,840.765305,NaN
7,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-21,8624.716645,7942.351660,NaN,898.576747,852.019934,NaN
8,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-22,10186.656074,8420.291157,NaN,1063.237706,900.654461,NaN
9,121b05df-f5f6-4029-92a7-5420dea45e4d,Koki HASEGAWA,2023-01-23,10051.948688,8504.948769,NaN,1041.028399,905.740394,NaN


In [29]:
# ============================================================
# 特徴量分類（ACWR / Dynamic / Static）
# ============================================================

acwr_cols = [
    c for c in daily_raw.columns
    if c.startswith("EWMA7_")
    or c.startswith("EWMA28_")
    or c.startswith("ACWR_EWMA_")
    or c.startswith("EWMA_ratio_")
]

dynamic_cols = [
    "max_vel",
    "mean_heart_rate",
    "max_heart_rate",
    "velocity_band4_total_distance",
    "velocity_band5_total_distance",
    "velocity_band6_total_distance",
    "heart_rate_band3_total_duration",
    "heart_rate_band4_total_duration",
    "acceleration_load",
    "ima_accel_high",
    "ima_decel_high",
    "ima_cod_left_high",
    "ima_cod_right_high",
]
dynamic_cols = [c for c in dynamic_cols if c in daily_raw.columns]

ban_cols = ["total_distance", "total_player_load"]

meta_cols = ["athlete_id", "athlete_name", "date_", "md_offset", "md_phase", "is_match_day"]

static_cols = [
    c for c in daily_raw.columns
    if c not in acwr_cols
    and c not in dynamic_cols
    and c not in ban_cols
    and c not in meta_cols
]

df_acwr    = daily_raw[meta_cols + acwr_cols].copy()
df_dynamic = daily_raw[meta_cols + dynamic_cols].copy()
df_static  = daily_raw[meta_cols + static_cols].copy()



In [30]:
# ===========================================
# 選手ごとのデータを保存（03 用）
# ===========================================

PLAYER_DIR = "/workspace/data/02/players"
os.makedirs(PLAYER_DIR, exist_ok=True)

for athlete_id, sub in daily_raw.groupby("athlete_id"):
    out_dir = f"{PLAYER_DIR}/{athlete_id}"
    os.makedirs(out_dir, exist_ok=True)

    df_dynamic[df_dynamic["athlete_id"] == athlete_id] \
        .to_parquet(f"{out_dir}/{athlete_id}_dynamic.parquet", index=False)

    df_static[df_static["athlete_id"] == athlete_id] \
        .to_parquet(f"{out_dir}/{athlete_id}_static.parquet", index=False)

    df_acwr[df_acwr["athlete_id"] == athlete_id] \
        .to_parquet(f"{out_dir}/{athlete_id}_acwr.parquet", index=False)

print("💾 Saved per-player features under /workspace/data/02/players/")



💾 Saved per-player features under /workspace/data/02/players/
